In [1]:
import requests as req

In [ ]:
a = req.get('https://www.msn.com/zh-tw/entertainment/news/%E5%A4%A7%E9%99%B8%E6%AD%8C%E6%89%8B%E9%84%AD%E8%88%88%E9%A6%96%E7%99%BBticc%E9%96%8B%E5%94%B1-%E6%86%B6%E5%8F%B0%E5%8C%97%E9%80%99%E5%A4%9C-%E4%B8%8D%E7%9F%A5%E8%A6%BA%E5%94%B1%E4%BA%8610%E5%B9%B4/ar-AA1FpnTZ?ocid=msedgntp&pc=U531&cvid=2b9f037f53484f3eabd4beb75c685c6f&ei=16', params=)

In [6]:
print(a.text)

<!DOCTYPE html>
<html lang="zh-tw" dir="ltr" >
<head data-info="f:msnallexpusers,prg-sp-liveapi,prg-wtch-sh-car-t2,prg-watch-kumo,prg-watch-scd,prg-wtch-sh-car,prg-wtch-sh-car-idx2,prg-wtch-shorts,prg-hp-fvbv,ads-cfv42-50,prg-views-respad,1s-wpo-lock-rtevbq,prg-1sw-sacfi2iuff,prg-1sw-sa-nrt_rcl_30_glb,prg-adspeek,btrecrow3,prg-pr2-layout3f,1s-wpo-pr2-cc-layout3,1s-wpo-pr2-layout,prg-pr2-deadloop,prg-pr2-gvfl3,prg-pr2-ovrisf,btie-msncons-c,1s-fcrypt,prg-1sw-sacc-unv9cc,prg-1sw-sacc-unv8t1,1s-ntf1-wswgdd,1s-ntf1-nbswgd,1s-ntf1-wswgd,prg-1sw-ntf-spmh,prg-upsaip-w1-t,1s-rpssecautht,chatn_v2_t1,1s-ntf2-c1nmfctl2,prg-pr2-ntf-fdltbruc,prg-pr2-ntf-ufnt,1s-logdb-badge-ufnt,1s-ntf2-ufndedup,prg-csacclink-c,prg-sh-rmitmlnk,nopinglancecardit,ads-nopostsq-t,ads-nopostsq,1s-uasdisf-t,ads-creativelog-c,ads-usepme,ads-anjson-migt,sh-bdvid,prg-sh-bd-video,d6aej896,ads-nooutbrain,ads-dable-migr,release-outlook-app,ads-prcrid-bi,cg-ab-testing-c,prg-1sw-tvid-t20,1s-p1-vid-vs,prg-1sw-tvid-p1,prg-1sw-videop